Analisis descriptivo 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df=pd.read_csv("ncr_ride_bookings.csv")

Establecer formato de fecha y hora, agregamos una columna con la hora de la reservación como un entero y una columna con el día de la semana

In [3]:
df["Date"] = pd.to_datetime(df["Date"])
df["Hour"] = df["Time"].str[:2].astype(int)
df["DayOfWeek"] = df["Date"].dt.day_name()

Agrupamos

In [ ]:
grouped = df.groupby(["DayOfWeek", "Hour"]).agg(
    total_rides=("Booking ID", "count"),
    driver_cancels=("Cancelled Rides by Driver", "sum"),
    customer_cancels=("Cancelled Rides by Customer", "sum")
).reset_index()